In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
from typing import Tuple, Optional, List
import pandas as pd
import time

In [131]:
from Webscrape import Webscrape

In [ ]:
base_url: str = 'https://www.zorgkaartnederland.nl'
facility: str = 'tandartsenpraktijk'
outputfile: str = '20250522\ -\ Testbestand\ Zorgkaart\ Nederland.xlsx'
scrape: Webscrape = Webscrape(base_url=base_url)

## Debug

In [133]:
max_page = 5

In [135]:
all_filter_result = []
for num in range(1, max_page + 1):
    print(f'ronde: {num}')
    temp_result = scrape.get_filter_result(facility=facility, page_id=num)
    all_filter_result.extend(temp_result)

ronde: 1
ronde: 2
ronde: 3
ronde: 4
ronde: 5


In [137]:
len(all_filter_result)

100

In [138]:
result_dict = scrape.get_filter_links(results=all_filter_result)
len(result_dict)

100

In [139]:
result_dict

{'Dent-S': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-dent-s-amsterdam-3033585',
 'MondCleanic, Center for Dental Excellence': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-mondcleanic-center-for-dental-excellence-noordwijk-3045043',
 'Tridenzo mondzorg': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-tridenzo-mondzorg-breda-3058852',
 "Tandheelkundig Centrum 't Veld": 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-tandheelkundig-centrum-t-veld-t-veld-3040694',
 'Praktijk voor Endodontologie Bussum': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-praktijk-voor-endodontologie-bussum-bussum-3036956',
 'Ittmann Tandartsen': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-ittmann-tandartsen-velp-3043194',
 'TandartsCraanen.nl': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-tandartscraanen-nl-ubbena-3020937',
 'Tandartsenpraktijk Quis

In [ ]:
result_list = []
for name, url in result_dict.items():
    temp_dict = {}
    temp_dict['naam'] = name
    temp_dict['type'] = facility.capitalize()
    print(f'Naam: {name}')
    info_dict = scrape.get_facility_info(facility_url=url)
    print(f'    info_dict: {info_dict}')
    temp_dict.update(info_dict)
    result_list.append(temp_dict)
    time.sleep(0.25)  

Naam: Dent-S
    info_dict: {'plaats': 'Amsterdam', 'postcode': '1079HW', 'straat': 'Rijnstraat', 'huisnummer': '246', 'toevoeging': 'H'}
Naam: MondCleanic, Center for Dental Excellence
    info_dict: {'plaats': 'Noordwijk', 'postcode': '2201HR', 'straat': 'Voorstraat', 'huisnummer': '107', 'toevoeging': None}
Naam: Tridenzo mondzorg
    info_dict: {'plaats': 'Breda', 'postcode': '4812XT', 'straat': 'Vijfhagen', 'huisnummer': '43', 'toevoeging': None}
Naam: Tandheelkundig Centrum 't Veld
    info_dict: {'plaats': "'t Veld", 'postcode': '1735HK', 'straat': 'Dreef', 'huisnummer': '2', 'toevoeging': 'A'}
Naam: Praktijk voor Endodontologie Bussum
    info_dict: {'plaats': 'Bussum', 'postcode': '1404JA', 'straat': 'Wilhelminaplantsoen', 'huisnummer': '15', 'toevoeging': None}
Naam: Ittmann Tandartsen
    info_dict: {'plaats': 'Velp', 'postcode': '6881VM', 'straat': 'President Kennedylaan', 'huisnummer': '1', 'toevoeging': 'A'}
Naam: TandartsCraanen.nl
    info_dict: {'plaats': 'Ubbena', 'po